In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering


In [3]:
train = pd.read_csv(r'D:\Project Phase II\Dataset\train.csv',encoding='cp1252')
test = pd.read_csv(r'D:\Project Phase II\Dataset\test.csv',encoding='cp1252')

In [4]:
todrop = ['Stime', 'Ltime', 'attack_cat']
reducedTrain = train.drop(todrop, axis = 1)
reducedTest = test.drop(todrop, axis = 1)

In [5]:
x_train = reducedTrain.drop(['Label'], axis = 1)
y_train = reducedTrain['Label']
x_test = reducedTest.drop(['Label'], axis = 1)
y_test = reducedTest['Label']

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Define the GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x, adj_matrix):
        x = F.relu(self.fc1(torch.matmul(adj_matrix, x)))
        x = self.fc2(torch.matmul(adj_matrix, x))
        return F.log_softmax(x, dim=1)

In [9]:
#Create a networkx directed graph from the dataframes
import networkx as nx
import numpy as np
import pandas as pd 

trainnodes = set()
testnodes = set()

for i in range(len(train)):
    trainnodes.add(str(train.iloc[i]['srcip'])+":"+str(train.iloc[i]['sport']))
    trainnodes.add(str(train.iloc[i]['dstip'])+":"+str(train.iloc[i]['dsport']))



In [11]:
for i in range(len(test)):
    testnodes.add(str(test.iloc[i]['srcip'])+":"+str(test.iloc[i]['sport']))
    testnodes.add(str(test.iloc[i]['dstip'])+":"+str(test.iloc[i]['dsport']))
    

In [12]:
trainG = nx.DiGraph()
for node in trainnodes:
    trainG.add_node(node)

testG = nx.DiGraph()
for node in testnodes:
    testG.add_node(node)


In [14]:
len(train.iloc[0][4:46])

42

In [17]:
train.iloc[0][3:48]

dsport                       83
proto                       120
state                         2
dur                    0.001046
sbytes                      146
dbytes                      178
sttl                         31
dttl                         29
sloss                         0
dloss                         0
service                       2
Sload                558317.375
Dload               680688.3125
Spkts                         2
Dpkts                         2
swin                          0
dwin                          0
stcpb                         0
dtcpb                         0
smeansz                      73
dmeansz                      89
trans_depth                   0
res_bdy_len                   0
Sjit                        0.0
Djit                        0.0
Stime                1424231464
Ltime                1424231464
Sintpkt                   0.003
Dintpkt                   0.001
tcprtt                      0.0
synack                      0.0
ackdat  

In [18]:
train.iloc[0][-1]

0

In [19]:
# add edges
for i in range(len(train)):
    trainG.add_edge(str(train.iloc[i]['srcip'])+":"+str(train.iloc[i]['sport']), str(train.iloc[i]['dstip'])+":"+str(train.iloc[i]['dsport']), weight = train.iloc[i][4:46], label=train.iloc[i][-1] )

In [21]:
adj_matrix = nx.adjacency_matrix(trainG)
adj_matrix = torch.FloatTensor(adj_matrix.toarray())

ValueError: object dtype is not supported by sparse matrices

In [7]:
#Pass networkx graph to GCN
import torch
import torch.nn as nn


# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(x_train)
y_train_tensor = torch.LongTensor(np.array(y_train))
X_test_tensor = torch.FloatTensor(x_test)
y_test_tensor = torch.LongTensor(np.array(y_test))

# Define the adjacency matrix (assuming no graph structure, so identity matrix)
num_nodes = X_train.shape[0]
adj_matrix = torch.eye(num_nodes)

# Instantiate GCN model
input_dim = X_train.shape[1]
hidden_dim = 4
output_dim = 1
model = GCN(input_dim, hidden_dim, output_dim)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.NLLLoss()

# Train the model
def train_model(model, optimizer, criterion, adj_matrix, X_train, y_train, epochs=100):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(X_train, adj_matrix)
        loss = criterion(output, torch.argmax(y_train, dim=1))
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

train_model(model, optimizer, criterion, adj_matrix, X_train_tensor, y_train_tensor)


ValueError: could not determine the shape of object type 'DataFrame'

In [ ]:
# Evaluate the model
def evaluate_model(model, X_test, y_test, adj_matrix):
    model.eval()
    with torch.no_grad():
        output = model(X_test, adj_matrix)
        _, predicted = torch.max(output, 1)
        correct = (predicted == torch.argmax(y_test, dim=1)).sum().item()
        accuracy = correct / y_test.size(0)
        print(f'Accuracy: {accuracy}')

evaluate_model(model, X_test_tensor, y_test_tensor, adj_matrix)